# Spiel Definition
In diesem Kapitel werden Funktionen diskutiert, die nötig sind, um das Spiel Mühle

$$G_{Nine Men's Morris} = 〈States,s_0,Players,nextStates,finished,utility〉$$

zu definieren. Es werden die [offizellen Tunierregeln](http://www.muehlespiel.eu/images/pdf/WMD_Turnierreglement.pdf) in der Version `3.0` vom 11. Mai 2019 des Weltmühlespiel Dachverbandes verwendet. Jedoch werden die Regeln für Unentschieden ausgelassen, da diese die Zustände zu groß machen würden. Eine Implementierung dieser Regeln ist im Kapitel über die GUI zu finden.

In [ ]:
from IPython.core.display import HTML
with open("style.html", "r") as file:
    css = file.read()
HTML(css)

Mit Hilfe des Magic Command `%run` werden Hilfsmethoden geladen, die in einem eigenen Kapitel beschrieben werden. 

In [ ]:
%run ./nmm-game-utils.ipynb

## Spieler

Zunächst soll die Menge der Spieler definiert werden. Da Mühle mit zwei Spielern gespielt wird, die jeweils weiße oder schwarze Steine legen, werden die Spieler äquivalent benannt.
* Der beginnende Spieler weiß wird als `w` dargestellt,
* der gegnerische Spieler als `b`.

Für die Implementierung der Funktionen ist diese Menge nicht nötig. Aus Gründen der Vollständigkeit soll diese hier dennoch definiert werden.

In [ ]:
Player = {'w', 'b'}

## Zustände
Die vollständige Menge aller Zustände ist aufwendig zu berechnen und wird ebenfalls nicht für die Implementierung benötigt. Da die Zustände berechnet werden, sobald diese benötigt werden. Deswegen wird hier auf eine Definition von $States$ verzichtet.

Der Startzustand $s_0$ hingegen wird zu Beginn des Spiel benötigt und ist im Folgenden definiert. Er bezeichnet ein leeres Spielbrett, bei dem beide Spieler noch alle ihre Steine legen müssen.
Ein Zustand (`state`) ist ein Tupel, das
1. den Stapel (`stash`) von zu legenden Steinen und
2. das Spielbrett (`board`) selbst
beinhaltet.

Der Stapel ist ein Tupel $\langle w, b \rangle$, das die Anzahl der zu legenden Steine von Weiß $w$ und Schwarz $b$ beinhaltet. Diese werden in der genannten Reihenfolge als Zahlen dargestellt.
$$w, b \in \{0...9\}$$

Das Spielbrett wird durch eine Tripel beschrieben, die die Ringe des Spielbretts beinhaltet. Alle zusammengehörigen Spielbrettpositionen, die sich auf dem gleichen Ring befinden, sind in der folgenden Abbildung durch eine Farbe markiert. Der äußere Ring ist $rot$, der mittlere $grün$ und der innere $blau$ dargestellt, damit gilt für einen Index $r$ der einen der Ringe bezeichnet $r \in \{0...2\}$.

![](../images/nmm-rings.png)

Die Spielpositionen (`cells`) $c$ auf den Ringen sind beginnend mit $c=0$ ab der oberen linken Ecke im Uhrzeigersinn durch nummeriert. Dadurch ergibt sich, dass $c \in \{0...7\}$. In der Abbilung sind die Koordinaten der Spielpositionen eingezeichnet, so liegt beispielsweise $\langle r,c \rangle = \langle 2,4 \rangle$ auf dem inneren Ring in der unteren rechten Ecke. An diesen Spielpositionen wird der Spieler $p$ oder eine leere Spielposition gespeichert, für diese gilt $p \in Player \cup \{'\ '\}$.

So ist ein State ein Tupel $\langle stash, board \rangle$ dessen Parameter
* `stash` ein weiteres Tupel, bestehend aus $\langle w, b \rangle$ und
* `board` ein Tripel, welches die Ringe $r_0$, $r_1$ und $r_2$ beinhaltet. Die Ringe selbst sind Neun-Tupel, dessen Elemente die einzelnen Spielpositionen $c \in Player \cup \{'\ '\}$ darstellen.

Der Startzustand $s_0$ wird in Python wie folgt geschrieben:

In [ ]:
s0 = ((9, 9), (
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '),
    (' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ')
))

## Folgezustände
Für die Definition des Spieles Mühle $G_{Nine Men's Morris}$ wird die Funktion `nextStates` benötigt. Diese nimmt einen Zustand `s` und einen Spieler `p` entgegen und errechnet mit diesen alle Folgezustände, die entstehen, wenn der gegebene Spieler einen legalen Zug spielt.

Da das Spiel in drei Phasen aufgeteilt ist, ist auch die Implementierung aus Gründen der Übersicht in die folgenden Phasen unterteilt:
1. Die erste Phase des Spiels heißt _placing_ Phase, in der die Spieler alle ihre Steine aus dem Stapel auf das Spielbrett legen müssen. Hierbei dürfen bereits Mühlen gelegt und geschlagen werden. Es muss ein Stein platziert werden.
2. Die zweite Phase heißt _moving_ Phase und beginnt sobald der Stapel des Spielers leer ist. Beide Spieler kommen gleichzeitig in die zweite Phase, in der die eigenen Steine nur noch entlang der Linien auf die nächste Spielposition geschoben werden dürfen. Es muss ein Stein bewegt werden.
3. Die letzte Phase heißt _flying_ Phase. Diese Phase beginnt für einen Spieler, sobald dieser nur noch $3$ Steine auf dem Spielbrett liegen hat und sein Stapel leer ist. Der Eintritt in die dritte Phase ist nicht vom gegnerischen Spieler abhängig, somit können Spieler für mehrere Züge alleine in der dritten Phase sein. Hierbei können die verbleibenden Steine beliebig bewegt werden, ohne dass die Positionen direkt nebeneinander liegen müssen. Es muss ein Stein bewegt werden.

In der ersten Phase, der _placing_ Phase, wird ein Stein vom Stapel genommen und auf ein freies Feld gelegt. Diese Züge werden in der Funktion `nextStatesPlace` implementiert, hierbei wird ein Spieler `p` und ein Zustand `s` erwartet, für den angenommen wird, dass sich der aktuelle Zustand in der ersten Phase befindet.
Die Rückgabe dieser Funktion ist die Menge aller Zustände, die erreicht werden können, in denen der Spieler `p` einen legalen Spielzug auf dem Zustand `s` ausführt.

1. Zunächst werden die Mühlen aus dem aktuellen Zustand mit Hilfe der Funktion `findMills` gespeichert.
2. Daraufhin wird auf jede freie Spielposition ein neuer Stein des Spielers `p` gelegt.
3. Nun werden unter Verwendung von dem Ergebnis aus _1._ und der Funktion `countNewMills` alle neu entstandenen Mühlen gezählt. Für diese Anzahl gilt $a \in \{0,1,2\}$.
4. Um für die neu entstandenen Mühlen die entsprechende Anzahl an Steinen zu schlagen, wird die Hilfsfunktion `poundMills` verwendet.
5. Zum Schluss wird vom Stapel des Spielers `p` ein Stein weg genommen und der Stapel mit den Spielbrettern wieder zu Zuständen vereint.

In [ ]:
def nextStatesPlace(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current mills the player has
    mills = findMills(board, p)

    # Place a stone in any empty cell
    placeBoards = {
        place(board, (r, c), p)
        for (r, c) in findEmptyCells(board)
    }
    # Calculate how many new mills were created
    boardMills = {
        board: countNewMills(board, mills, p)
        for board in placeBoards
    }

    # Here all final boards will be collected
    boards = {
        result
        for (b, count) in boardMills.items() 
        for result in poundMills(b, count, p)
    }

    # Remove one stone from the players stache
    (cw, cb) = (cw-1, cb) if p == 'w' else (cw, cb-1)

    # Return all possible states
    return { ((cw, cb), board) for board in boards }

In der _moving_ Phase (2) wird ein Stein des aktuellen Spielers entlang der Linien zu einer benachbarten, leeren Spielposition geschoben. Die Definition der Funktion `nextStatesMove` wird wieder eine Zustand `s` und ein Spieler `p` erwartet, die sich in der zweiten Phase befinden.
Die Rückgabe dieser Funktion ist die Menge aller Zustände, die erreicht werden können, in denen der Spieler `p` einen legalen Spielzug auf dem Zustand `s` ausführt.

1. Zunächst werden die Mühlen aus dem aktuellen Zustand mit Hilfe der Funktion `findMills` gespeichert.
2. Danach werden die Positionen aller Steine des Spielers `p` und dessen leere Nachbarpositionen mit Hilfe der Funktionen `findCellsOf` und `findNeighboringEmptyCells` errechnet. Die Steine werden dort hin bewegt.
3. Nun werden unter Verwendung von dem Ergebnis aus _1._ und der Funktion `countNewMills` alle neu entstandenen Mühlen gezählt. Für diese Anzahl gilt $a \in \{0,1\}$.
4. Um für die neu entstandenen Mühlen die entsprechende Anzahl an Steinen zu schlagen, wird die Hilfsfunktion `poundMills` verwendet.
5. Zum Schluss wird der Stapel mit den Spielbrettern wieder zu Zuständen vereint.

In [ ]:
def nextStatesMove(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current mills the player has
    mills = findMills(board, p)

    # Choose any stone of the player and move it to an empty neighbor
    moveBoards = {
        move(board, src, des)
        for src in findCellsOf(board, p)
        for des in findNeighboringEmptyCells(board, src)
    }

    # Calculate how many new mills were created
    boardMills = {
        b: countNewMills(b, mills, p)
        for b in moveBoards
    }

    # Here all final boards will be collected
    boards = {
        result
        for (b, count) in boardMills.items() 
        for result in poundMills(b, count, p)
    }

    return { ((cw, cb), board) for board in boards }

Die letzte Hilfsfunktion `nextStatesFly` wird in der Phase 3 verwendet und erwartet äquivalent zu den anderen Hilfsmethoden einen Zustand `s` und Spieler `p` in der Phase 3. Hier wird ein Stein des Spieler `p` an eine andere Position auf dem Spielbrett bewegt. Diese Position muss keine Nachbarposition sein und Steine können übersprungen werden.
Die Rückgabe dieser Funktion ist die Menge aller Zustände, die erreicht werden können, in denen der Spieler `p` einen legalen Spielzug auf dem Zustand `s` ausführt.

1. Zunächst werden die Mühlen aus dem aktuellen Zustand mit Hilfe der Funktion `findMills` gespeichert.
2. Danach werden die Positionen aller Steine des Spielers `p` und dessen leere Nachbarpositionen mit Hilfe der Funktionen `findCellsOf` und `findNeighboringEmptyCells` errechnet. Die Steine werden dort hin bewegt.
3. Nun werden unter Verwendung von dem Ergebnis aus _1._ und der Funktion `countNewMills` alle neu entstandenen Mühlen gezählt. Für diese Anzahl gilt $a \in \{0,1\}$.
4. Um für die neu entstandenen Mühlen die entsprechende Anzahl an Steinen zu schlagen, wird die Hilfsfunktion `poundMills` verwendet.
5. Zum Schluss wird der Stapel mit den Spielbrettern wieder zu Zuständen vereint.

In [ ]:
def nextStatesFly(s, p):
    # Extract the count of the stones and the board
    ((cw, cb), board) = s
    # Calculate all current mills the player has
    mills = findMills(board, p)

    # Choose any stone of the player and move it to an empty neighbor
    moveBoards = {
        move(board, src, des)
        for src in findCellsOf(board, p)
        for des in findEmptyCells(board)
    }

    # Calculate how many new mills were created
    boardMills = {
        b: countNewMills(b, mills, p)
        for b in moveBoards
    }

    # Here all final boards will be collected
    boards = {
        result
        for (b, count) in boardMills.items() 
        for result in poundMills(b, count, p)
    }

    return { ((cw, cb), board) for board in boards }

Die Implementierung der `nextStates` Funktion führt nun die vorher definierten Funktionen zusammen. Die aktuelle Phase für den gegebenen Zustand `s` und Spieler `p` wird mit der Hilfsfunktion `playerPhase` errechnet und auf Grund dessen eine Fallunterscheidung ausgeführt, sodass gilt

$$
nextStates(s, p) = \begin{cases}
nextStatesPlace(s, p) & falls & playerPhase(s, p) = 1 \\
nextStatesMove(s, p) & falls & playerPhase(s, p) = 2 \\
nextStatesFly(s, p) & falls & playerPhase(s, p) = 3
\end{cases}
$$

In [ ]:
def nextStates(s, p):
    phase = playerPhase(s, p)
    if phase == 1:
        return nextStatesPlace(s, p)
    elif phase == 2:
        return nextStatesMove(s, p)
    else:
        return nextStatesFly(s, p)

## Spielende

Für die Definition des Spieles Mühle $G_{Nine Men's Morris}$ werden zwei weitere Funktionen benötigt, die das Ende des Spiels behandeln: `finished` und `utility`.

Die Funktion `finished` errechnet für einen gegebenen Zustand `s`, ob das Spiel beendet ist, wenn Spieler `p` an der Reihe ist. Dies ist der Fall, g.d.w.
* einer der Spieler $p \in Players$ weniger als 3 Steine hat (`hasEnoughStones`) oder
* der Spieler `p` keinen legalen Zug mehr tätigen kann.

Der optionale Parameter `ns` ist lediglich eine Optimierung, die es ermöglicht, bereits berechnete Folgezustände zu verwenden, anstatt diese noch einmal errechnen lassen zu müssen.

In [ ]:
def finished(s, p, ns=None):
    if not ns:
        ns = nextStates(s, p);
    return not hasEnoughStones(s, 'w') or \
           not hasEnoughStones(s, 'b') or \
           len(ns) == 0

Die Funktion `utility` errechnet für den gegebenen Zustand `s` und Spieler `p`, falls $finished(s, p) = true$ gilt, wer gewonnen hat.

$$
utility(s, p) = \begin{cases}
-1 & falls & p \ verliert \ in \ s \\
 0 & falls & Unentschieden         \\
 1 & falls & p \ gewinnt  \ in \ s
\end{cases}
$$

Sobald ein Spieler zu wenig Steine hat (`hasEnoughStones`) oder keinen legalen Zug mehr tätigen kann, hat dieser verloren. Dadurch gewinnt automatisch der gegnerische Spieler. Zwar gibt es im Spiel Mühle ein Unentschieden, dies kann aber nicht anhand der Zustände erkannt werden, da hierbei die vorher gespielten Spielzüge betrachtet werden müssen. 

Äquivalent zu der Funktion `finished` ist Parameter `ns` optional und stellt lediglich eine Optimierung dar, die es ermöglicht, bereits berechnete Folgezustände zu verwenden, anstatt diese noch einmal errechnen lassen zu müssen.

In [ ]:
def utility(s, p, ns=None):
    if not ns:
        ns = nextStates(s, p);
     
    if not hasEnoughStones(s, p):
        return -1
    if not hasEnoughStones(s, opponent(p)):
        return 1
    if len(nextStates(s, p)) == 0:
        return -1

    # Should be impossible, as utility() will only be called if finished() returns True
    return 0